In [1]:
# IMPORT Thư viện
#  pip install numpy pandas jupyter notebook openpyxl xlrd
import pandas as pd
import os
import openpyxl
import xlrd
import numpy as np

In [2]:
# IMPORT CÁC FILE (5 FILES): TỔNG NHẬP XUẤT TỒN ; GIÁ BÁN; MUA HÀNG ; BÁN HÀNG; INPUT

# IMPORT 1 Tổng hợp nhập xuất tồn -> RENAME 
Data_Ton = pd.read_excel("Tổng hợp Nhập - Xuất - Tồn YTD.xls",usecols="A,F,K:L",skipfooter=1,header=5)
Data_Ton=Data_Ton.rename(columns={"Unnamed: 0":"ID",
                                "Unnamed: 5":"ID_Category",
                                "Số lượng.2":"Tổng xuất", 
                                "Giá trị.2":"Giá trị xuất" })

#Import INPUT Quotation
Data_Request=pd.read_excel("Input.xlsx",sheet_name='Data', usecols="B:I")


#Import 2: Báo cáo giá bán tồn kho
Data_GiaBanTonKho=pd.read_excel("Báo cáo giá bán, tồn kho.xls",usecols="C,F,J",header=4,skipfooter=1)
Data_GiaBanTonKho=Data_GiaBanTonKho.rename(columns={"Mã số":"ID"})
Data_GiaBanTonKho.dropna(how='all',inplace=True)


#Import 3: Sổ chi tiết bán hàng
Data_BanHang=pd.read_excel("Sổ chi tiết bán hàng.xls",usecols="F,M",skipfooter=1,header=6)
Data_BanHang=Data_BanHang.rename(columns={"Unnamed: 0":"Ngày bán",
                                        "Unnamed: 5":"ID" ,
                                        "Giá bán":"Giá bán giảm"})


#Import 4: Sổ chi tiết mua hàng
Data_MuaHang=pd.read_excel("Sổ chi tiết mua hàng (2).xls",usecols="A,C,F,I:J",header=5,skipfooter=1)
Data_MuaHang=Data_MuaHang.rename(columns={"Unnamed: 0":"Ngày nhập",
                                        "Unnamed: 2":"Nhà cung cấp",
                                        "Unnamed: 5":"ID",
                                        "Số lượng":"Tổng nhập",
                                        "Đơn giá":"Đơn giá nhập"})


In [3]:
# TẠO BỘ LỌC CHO SỔ MUA HÀNG: GIÁ NHẬP NHỎ NHẤT; NGÀY NHẬP GẦN NHẤT
# # FILTER: Đơn giá nhập >0; Tổng nhập > 0; 
# Data_MuaHang=Data_MuaHang.loc[(Data_MuaHang['Đơn giá nhập']>0) & (Data_MuaHang['Tổng nhập']>0)].sort_values('Đơn giá nhập')
# # Tạo giá nhập nhỏ nhất
# Min_value=Data_MuaHang.groupby('ID')['Đơn giá nhập'].min()
# Data_MuaHang=Data_MuaHang.merge(Min_value,on='ID',suffixes=('', '_min'))
# # -> Filter theo ID có đơn giá nhập nhỏ nhất
# Data_MuaHang = Data_MuaHang[Data_MuaHang['Đơn giá nhập']==Data_MuaHang['Đơn giá nhập_min']].drop('Đơn giá nhập_min', axis=1)

#Backup
Data_MuaHang_Min=Data_MuaHang
#-> Filter Mã hàng có ngày nhập gần nhất

Date_max=Data_MuaHang.groupby('ID')['Ngày nhập'].max()
Data_MuaHang=Data_MuaHang.merge(Date_max,on='ID',suffixes=('', '_max'))
# -> Filter theo ID có đơn giá nhập nhỏ nhất
Data_MuaHang = Data_MuaHang[Data_MuaHang['Ngày nhập']==Data_MuaHang['Ngày nhập_max']].drop('Ngày nhập_max', axis=1)


# Data_MuaHang.loc[Data_MuaHang['ID']=="90913-02105"]
# Data_MuaHang.loc[Data_MuaHang['ID']=="T24NC50WB"]

In [13]:
# TẠO BỘ LỌC CHO NHÀ CUNG CẤP NHỎ NHẤT

#-> Filter Mã hàng có ngày nhập gần nhất

DonGia_min=Data_MuaHang_Min.groupby('ID')['Đơn giá nhập'].min()
Data_MuaHang_Min=Data_MuaHang_Min.merge(DonGia_min,on='ID',suffixes=('', '_min'))
# -> Filter theo ID có đơn giá nhập nhỏ nhất
Data_MuaHang_Min = Data_MuaHang_Min[Data_MuaHang_Min['Đơn giá nhập']==Data_MuaHang_Min['Đơn giá nhập_min']].drop('Đơn giá nhập_min', axis=1)


,Ngày nhập,Nhà cung cấp,ID,Tổng nhập,Đơn giá nhập
12,2022-08-27,NCC-THAILAND,48510-0D470,30,0.00
21,2020-04-17,NK-FPI,48510-80133,6,917421.50
40,2020-04-17,NK-FPI,48530-8Z007,7,1023430.57
44,2022-05-13,NK-CHI,48540-09691,26,1736046.00
51,2021-01-12,HTAUTOHN,48815-06200,10,33000.00
...,...,...,...,...,...
14196,2020-02-11,HTAUTOHN,87103-48080,1,2847000.00
14197,2020-02-11,HTAUTOHN,15163-28010,3,42000.00
14198,2020-02-11,HTAUTOHN,53112-02630,1,387000.00
14199,2022-03-10,HTAUTOHN,17610-0C040,1,4950000.00


In [4]:
# TẠO BỘ LỌC CHO SỔ BÁN HÀNG: GIÁ GIẢM NHỎ NHẤT;
Data_BanHang = Data_BanHang[Data_BanHang['Giá bán giảm'] == Data_BanHang.groupby('ID')['Giá bán giảm'].transform('min')]
Data_BanHang = Data_BanHang.drop_duplicates(['ID','Giá bán giảm'], keep="first")


In [5]:
# TẠO BỘ LỌC CHO NHÀ SẢN XUẤT: GIÁ NHỎ NHẤT;

#-> Filter Mã hàng có ngày nhập gần nhất
UnitPrice_Min=Data_Request.groupby(['ID'])['Unit price'].min()
Data_Request=Data_Request.merge(UnitPrice_Min,on='ID',suffixes=('', '_min'))
# -> Filter theo ID có đơn giá nhập nhỏ nhất
Data_Request = Data_Request[Data_Request['Unit price']==Data_Request['Unit price_min']].drop('Unit price_min', axis=1)
# Data_Request_2.loc[Data_Request_2['ID']=='04152-YZZA4']


In [6]:
# TÍNH TOÁN GIÁ VỐN
# Calcuation on Tong Hop Ton Kho -------------------------------------------

# Gia Von= Gia Tri Xuat/SL Xuat * 1.05
Data_Ton['Giá vốn']=np.around(Data_Ton['Giá trị xuất']/Data_Ton['Tổng xuất']*1.05,0)


In [7]:
# TỔNG HỢP DỮ LIỆU

# Merge Request -> Gia Von
Data_Request= pd.merge(Data_Request,Data_Ton[['ID','Giá vốn','Tổng xuất']],on='ID', how='left')
# # Merge  (Báo cáo nhập xuất tồn)
Data_Request=pd.merge(Data_Request,Data_GiaBanTonKho[['ID','Giá bán','Tổng tồn']],how='left',on='ID')
# # # # Merge Giá bán hàng
Data_Request=pd.merge(Data_Request,Data_BanHang,how='inner',on='ID')
# # Merge Giá mua hàng
Data_Request=pd.merge(Data_Request,Data_MuaHang,how='left',on='ID')


In [8]:
# TÍNH TOÁN CÁC HỆ SỐ
# Calcuation on Request -------------------------------------------


#VND/US=24,000; he so = 1.13

Data_Request['Unit Price VND'] = np.around(Data_Request['Unit price']*24000*1.13,0)
Data_Request['Tỷ lệ Giá vốn chênh lệch'] = np.around((Data_Request['Unit Price VND']-Data_Request['Giá vốn'])/Data_Request['Giá vốn'],2)

In [9]:
# MẶC CẢ GIÁ VỚI NHÀ CUNG CẤP; RATE = 5%
RateDeal=0.05
CostTrans=0.13
RateVNDUSD=24000

# GIÁ MẶC CẢ VND

# Chiết khấu = 5%; Tổng chi phí = 13%

# Giá mặc cả VND :
# 	Khi tỉ lệ chênh lệch Unit Price VND/Giá vốn <=10% : Đơn giá nhập
# 	Khi tỉ lệ chênh lệch Unit Price VND / Giá vốn >= 10%
# 		Giá mặc cả = Đơn giá nhập * (1-Chiết khấu)
# Giá mặc cả USD:
# 	Giá mặc cả VND * VND/USD * 1/(1+Chi phí)
# 	= GIá mặc cả VND * 24000 / (1+13%)

# Giá vốn mới =
# 	Giá mặc cả USD / 24000

Data_Request['Giá mặc cả VND']=np.where(Data_Request['Đơn giá nhập']<=0.0,Data_Request['Đơn giá nhập']*(1-RateDeal),1)
Data_Request['Giá mặc cả VND']=np.where(Data_Request['Tỷ lệ Giá vốn chênh lệch']>=0.1,
                                Data_Request['Đơn giá nhập']*(1-RateDeal),
                                Data_Request['Đơn giá nhập'])


#GIÁ MẶC CẢ USD

Data_Request['Giá mặc cả USD']=np.around(Data_Request['Giá mặc cả VND']*1/RateVNDUSD*1/(1+CostTrans),2)
Data_Request['Giá vốn mới']=np.around(Data_Request['Giá mặc cả USD']*24000,0)
 


In [10]:
#SHOW KET QUA
# Data_Request_Show=Data_Request.iloc[:,[0,1,2,3,4,5,7,6,17,8,18,10,12,13,14,16,13,15,9,11,19,20,21]]
#ID,Unit Price, Unit VND, Giá vốn, %chênh lệch VND, Giá bán, giá giảm, Ngày nhập, Nhà cung cấp, Đơn giá nhập, tổng nhập, tổng xuất, tổng tồn, Giá mặc cả VND, Giá mặc cả USD
# Data_Request_Show=Data_Request.sort_values(by='Tổng nhập', ascending=False).reset_index(drop=True)
# Data_Request.loc[Data_Request['ID']=="T24VG4WS"]

Data_Show=Data_Request.iloc[:,[0,1,2,3,4,5,7,6,17,8,18,10,12,13,14,16,13,15,9,11,19,20,21]]


In [11]:
# EXPORT FILE DATA
with pd.ExcelWriter('Report.xlsx') as writer:
    Data_Request.to_excel(writer, sheet_name='Data')
    Data_Show.to_excel(writer,sheet_name='Show')
